In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model

import numpy as np 
import matplotlib.pyplot as plt

In [3]:
# 생성자 모델을 만듭니다.
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation=LeakyReLU(0.2)))
generator.add(BatchNormalization())
generator.add(Reshape((7,7,128)))
generator.add(UpSampling2D())
generator.add(Conv2D(64, kernel_size=5,padding='same'))
generator.add(BatchNormalization())
generator.add(Activation(LeakyReLU(0.2)))
generator.add(UpSampling2D())
generator.add(Conv2D(1,kernel_size=5, padding='same',activation='tanh'))



In [4]:
#판별자 모델을 만듭니다.

discriminator =Sequential()
discriminator.add(Conv2D(64,kernel_size=5, strides=2, input_shape=(28,28,1),padding='same'))
discriminator.add(Activation(LeakyReLU(0.2)))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1,activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy',optimizer='adam')
discriminator.trainble=False

In [5]:
#생성자와 판별자 모뎅을 연결시키는 gan모델을 만듭니다.

ginput = Input(shape=(100,)) #생성자에 집어넣을 거 생성 ?
dis_output =discriminator(generator(ginput)) #생성자의 실행결과를 판별자에 집어넣어 실행하고 결과값을 dis_output 저장 
gan = Model(ginput,dis_output) # 새로운 gen 모델 생성 ! 
gan.compile(loss='binary_crossentropy',optimizer ='adam') 
gan.summary() # 구조확인을 위해 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_1 (Sequential)   (None, 28, 28, 1)         865281    
                                                                 
 sequential_2 (Sequential)   (None, 1)                 14209     
                                                                 
Total params: 879,490
Trainable params: 866,818
Non-trainable params: 12,672
_________________________________________________________________


In [ ]:
#신경망을 실행시키는 함수를 만듭니다.
def gan_train(epoch,batch_size, saving_interval):
  
# MNIST 데이터 불러오기
    # 앞서 불러온 MNIST를 다시 이용합니다. 테스트 과정은 필요 없고
    # 이미지만 사용할 것이기 때문에 X_train만 호출합니다.
  (X_train, _),(_,_) = mnist.load_data()
  X_train = X_train.reshape(X_train.shape[0],28,28,1).astype('float32')

  #127.5를 뺀 후 127.5 로 나누어서 -1~1 사이의 값으로 바꿉니다.
  X_train = (X_train - 127.5)/127.5
  true = np.ones((batch_size,1))
  fake = np.zeros((batch_size,1))

  for i in range(epoch):
    #실제 테이터를 판별자에 입력하는 부분입니다.
    idx = np.random.randint(0, X_train.shape[0],batch_size)
    imgs = X_train[idx]
    d_loss_real = discriminator.train_on_batch(imgs,true)
    #가상 이미지를 판별자에 입력하는 부분입니다.
    noise = np.random.normal(0,1,(batch_size,100))
    gen_imgs = generator.predict(noise)
    d_loss_fake = discriminator.train_on_batch(gen_imgs,fake)
    #판별자와 생성자의 오차를 계산합니다.
    d_loss= 0.5 *np.add(d_loss_real,d_loss_fake)
    

In [6]:
(X_train, _),(_,_) = mnist.load_data()

In [7]:
X_train.shape[0]

60000